# மண் பரிசோதனை - Namma Mann Soil Analysis Demo

React-ready form → POST to `/analyze` → Chart + Tamil recommendations.
**Start backend:** `uvicorn backend.main:app --reload` from project root.

In [ ]:
import requests
import ipywidgets as widgets
from IPython.display import display, HTML

API = "http://localhost:8000"

def on_submit(_):
    data = {
        "pH": float(ph_in.value),
        "moisture": float(moist_in.value),
        "N": float(n_in.value),
        "P": float(p_in.value),
        "K": float(k_in.value),
    }
    r = requests.post(f"{API}/analyze", json=data)
    out = r.json()
    result.value = f"<b>Recommendation:</b> {out['recommendation']}<br><b>மொழி:</b> {out.get('tamil_recommendation','')}<br><b>ML Score:</b> {out['ml_score']}"

ph_in = widgets.FloatSlider(6.0, 3.5, 10, 0.1, description="pH")
moist_in = widgets.FloatSlider(50, 0, 100, 1, description="Moisture%")
n_in = widgets.FloatText(20, description="N ppm")
p_in = widgets.FloatText(15, description="P ppm")
k_in = widgets.FloatText(40, description="K ppm")
btn = widgets.Button(description="மண் பரிசோதனை")
result = widgets.HTML(value="")
btn.on_click(on_submit)

display(widgets.VBox([
    widgets.HTML("<h2 style='color:#059669'>மண் பரிசோதனை - Soil Test</h2>"),
    ph_in, moist_in, n_in, p_in, k_in,
    btn, result
]))

## React Component (for Vite/React app)

Copy this into your `src/components/SoilAnalysisForm.jsx`:

In [ ]:
react_code = '''
// SoilAnalysisForm.jsx - Mobile-first, Tamil/English toggle
import { useState } from "react";
import { BarChart, Bar, XAxis, YAxis, Tooltip } from "recharts";

const API = "http://localhost:8000";
const TAMIL = { title: "மண் பரிசோதனை", submit: "பரிசோதனை" };
const ENG = { title: "Soil Test", submit: "Analyze" };

export default function SoilAnalysisForm() {
  const [lang, setLang] = useState("ta");
  const [soil, setSoil] = useState({ pH: 6.5, moisture: 50, N: 25, P: 15, K: 40 });
  const [result, setResult] = useState(null);

  const t = lang === "ta" ? TAMIL : ENG;

  const submit = async () => {
    const r = await fetch(`${API}/analyze`, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(soil),
    });
    setResult(await r.json());
  };

  const chartData = result ? [
    { name: "pH", value: soil.pH, fill: "#10b981" },
    { name: "N", value: soil.N / 50, fill: "#059669" },
    { name: "P", value: soil.P / 20, fill: "#10b981" },
    { name: "K", value: soil.K / 100, fill: "#059669" },
  ] : [];

  return (
    <div className="min-h-screen bg-green-50 p-4" style={{ fontFamily: "sans-serif" }}>
      <h1 className="text-3xl font-bold" style={{ color: "#059669" }}>{t.title}</h1>
      <button onClick={() => setLang(lang === "ta" ? "en" : "ta")} className="mt-2 text-green-700">
        {lang === "ta" ? "English" : "தமிழ்"}
      </button>
      <div className="grid grid-cols-2 gap-4 mt-4">
        {["pH","moisture","N","P","K"].map(k => (
          <label key={k} className="block">
            <span className="text-green-800">{k}</span>
            <input type="number" value={soil[k]} onChange={e => setSoil({...soil,[k]: +e.target.value})}
              className="border border-green-300 rounded p-2 w-full" />
          </label>
        ))}
      </div>
      <button onClick={submit} className="mt-6 px-6 py-3 rounded bg-green-600 text-white font-bold">
        🌾 {t.submit}
      </button>
      {result && (
        <div className="mt-6 p-4 bg-white rounded shadow">
          <p className="text-green-800">{result.recommendation}</p>
          <p className="text-gray-600">{result.tamil_recommendation}</p>
          <p className="text-sm">ML Score: {result.ml_score}</p>
          {chartData.length && <BarChart width={300} height={200} data={chartData}><XAxis/><YAxis/><Tooltip/><Bar dataKey="value" /></BarChart>}
        </div>
      )}
    </div>
  );
}
'''
print(react_code)